### RAGを使って一問一答
1. Model読み込み
2. PromptTemplateの設定
3. FAISSのvector dataを取得
4. 類似ドキュメントの取得
5. 解答を得る

In [2]:
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import (
    AzureOpenAIEmbeddings,
    OpenAIEmbeddings,
    AzureChatOpenAI,
    ChatOpenAI
)
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
load_dotenv('../.env')

True

#### 1. Model読み込み

In [23]:
# embeddingsのModelを取得
embeddings = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="text-embedding-3-small",
        openai_api_version="2023-05-15",
    )
else:
    print("APIKeyの設定を確認してください")

# chatのモデルを取得
model = None
if os.getenv('AZURE_OPENAI_API_KEY') != "":
    # Azureの場合
    model = AzureChatOpenAI(
        azure_deployment="gpt-4o",
        openai_api_version="2024-12-01-preview"
    )
else:
    print("APIKeyの設定を確認してください")

#### 2.PromptTemplateの設定

In [5]:
system_prompt = (
    "あなたは質問対応のアシスタントです"
    "質問に答えるために、検索された文脈の以下の部分を使用してください。"
    "答えが分からない場合は、分からないと答えてください。"
    "回答は3分以内で簡潔にしてください。"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [6]:
prompt.invoke({"context": "今日の料理はカレーです", "input": "今日の料理はなんですか？"})

ChatPromptValue(messages=[SystemMessage(content='あなたは質問対応のアシスタントです質問に答えるために、検索された文脈の以下の部分を使用してください。答えが分からない場合は、分からないと答えてください。回答は3分以内で簡潔にしてください。\n\n今日の料理はカレーです'), HumanMessage(content='今日の料理はなんですか？')])

#### 3. FAISSのvector dataを取得

In [7]:
vectorstore = FAISS.load_local("./db", embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever()

#### 4. 類似ドキュメントの取得

In [8]:
relavant_docs = retriever.invoke("LLMとは何ですか？概要と特徴を教えてください。", k=3)

In [10]:
type(relavant_docs)

list

In [12]:
# 類似文書を表示
for doc in relavant_docs:
    print(doc)
    print("-----")

page_content='が与えられると、⽂書検索ツールが呼び出され、もっとも関連性が⾼い⽂書が取得される（通常、初めにク
エリと⽂書をベクトルで符号化し、次にクエリベクトルにユークリッドノルムで最も近いベクトルを持つ⽂
書を検索する）。その後、 LLM は、クエリと取得した⽂書の両⽅に基づいて出⼒を⽣成する[50]。
LLM は⾔語モデルであり、それ⾃体は⽬標を持たないためエージェントではないが、知的エージェントの構
成要素として使⽤することができる。
ReAct （ Reason + Act ）法は、 LLM をプランナーとして使⽤し、 LLM からエージェントを構築するものであ
る。 LLM は「考えごとを声に出して⾔う」よう促される。具体的には、⾔語モデルに対して、環境のテキス
ト表現、⽬標、可能な⾏動のリスト、および過去の⾏動と観察の記録が与えられる。 LLM は、⾏動を決める
前に 1 つまたは複数の思考を⾏い、それが環境内で実⾏される[51]。 LLM プランナーに与えられる環境の⾔語
的記述は、ときには環境を記述した論⽂のLaTeXコードすら考えられる[52]。
強化学習によるファインチューニング
ツールの使⽤
エージェントリフレクション法 は、いくつかのエピソ ドにわたって学習するエ ジェントを構築する⼿法である。各
エピソードの終わりに、 LLM はそのエピソードの記録が渡され、次のエピソードでより良い成績を出すため
の「教訓」を考えるように促される。これらの「教訓」は次のエピソードでエージェントに渡される。
モンテカルロ⽊探索では、 LLM をロールアウトのためのヒューリスティクスとして使⽤することができる。
プログラムされた世界モデルが利⽤できない場合、 LLM は世界モデルとして動作するように環境を説明する
よう促されることもある[54]。
オープンエンド探索では、 LLM を観測値の「興味深さ（ interestingness ）」のスコアリングに使⽤し、これを
通常の（⾮ LLM ）強化学習エージェントを誘導する報酬信号として使⽤することができる[55]。あるいは、
LLM に、カリキュラム学習のために次第に難しくなるタスクを提案させることもできる[56]。 LLM プランナ'
-----
page_content='る。この場合、⼈

In [17]:
relavant_docs[2]

Document(page_content='機械的解釈可能性  は、 LLM によって実⾏される推論を近似する記号アルゴリズムを発⾒することにより、\nLLM をリバースエンジニアリングすることを⽬的としている。オセロ GPT （ Othello-GPT ）はその⼀例で、オ\nセロの正当な⼿を予測するように⼩規模なTransformerが訓練された。その結果、オセロ盤の線形表現が存\n在し、この表現を変更することで、予測される正当なオセロの⼿が正しい⽅向に変化することがわかっ\nた[70][71]。別の例では、著者はモジュラ算術加算に対して⼩規模な Transformer を訓練し、得られたモデルを\nリバースエンジニアリングしたところ、離散フーリエ変換を使⽤していることがわかった[72]。\n別の例では、⼩規模な Transformer をKarel プログラムに対して訓練している。オセロ GPT の例と同様に、\nKarel プログラムのセマンティクスには線形表現があり、その表現を修正すると出⼒が正しく変更される。こ\nのモデルはまた、訓練セット内のプログラムよりも平均して短く、正しいプログラムを⽣成した[73]。\n2022 年の調査で、（チューニングされていない） LLM が、「⾃然⾔語を何らかの⾃明でない意味で理解できる\n（ことがある）か」という問いに対して、⾃然⾔語処理研究者の意⾒は真っ⼆つに分かれた[68]。「 LLM は理\n解⼒を持つ」派の⽀持者は、数学的推論のようないくつかの LLM の能⼒は、特定の概念を「理解」する能⼒\nを意味すると考えている。マイクロソフトのチームは、 2023 年に、 GPT-4 は「数学、コーディング、視覚、\n医学、法律、⼼理学などにまたがる斬新で難しいタスクを解決できる」とし、 GPT-4 は「汎⽤⼈⼯知能シス\nテムの初期バージョン（しかしまだ未完成）とみなすのが妥当だろう」と主張し、「ソフトウェア⼯学の受験\n者の試験に合格するシステムが、本当の意味で知的ではないと⾔えるだろうか？[74][75]」と述べた。 LLM を\n「地球外⽣命の知能」と呼ぶ研究者もいる[76][77]。たとえば、 Conjecture の CEO であるコナー・リーヒー\nは、チューニングされていない LLM を、まるで得体の知れ

#### 5. 解答を得る

In [24]:
chain = prompt | model
response = chain.invoke({"context": relavant_docs, "input": "LLMとは何ですか？概要と特徴を教えてください。"})

In [25]:
response.content

'LLM（Large Language Model）は、大規模な自然言語処理モデルを指します。これらのモデルは、膨大なテキストデータを基に機械学習技術を活用して訓練され、人間の言語を理解し生成する能力を持っています。\n\n### 概要\n- **目的**: 主にテキスト生成、質問応答、翻訳、要約などの自然言語処理タスクを実行します。\n- **基盤技術**: Transformerアーキテクチャを利用しており、深層学習技術によって構築されています。\n- **学習方法**: 大量のテキストデータを基に、文脈を理解し次の単語を予測する形で学習します。\n\n### 特徴\n1. **大規模性**: 数十億〜数千億のパラメータを持ち、大量のデータを扱える。\n2. **汎用性**: 自然言語理解・生成を含む広範なタスクに適用可能。\n3. **知的エージェントの構成要素**: 言語モデルとして、エージェントやツールとの連携が可能。\n4. **ファインチューニング**: 特定のタスクやツールに応じて性能を向上させるカスタマイズが可能。\n5. **計算資源の必要性**: 訓練や運用には高性能なコンピューティングリソースが必要。\n\nLLMは、単なる言語モデル以上に多様な応用があり、例えば検索拡張生成（RAG）や知的エージェント構築に利用されるなど、広範な可能性を秘めています。'